In [2]:
# 결측치 처리하기 : 드롭과 채우기
    # 데이터를 처리하려 할 때 데이터가 없는 경우는 매우 흔함
        # 이러한 경우 어떻게 처리하는 것이 적절한지에 대한 전략을 세워야 함
    # 가장 기본적인 전략은 데이터를 삭제하거나 채우는 방식
        # - 데이터가 없으면 해당 행이나 열을 삭제
        # - 평균값, 최빈값, 중간값 등으로 데이터를 채움

In [3]:
    # 예시 1
import numpy as np
import pandas as pd

raw_data = {'first_name' : ['Jason', np.nan, 'Tina', 'Jake', 'Amy'],
            'last_name' : ['Miller', np.nan, 'Ali', 'Milner', 'Cooze'],
            'age' : [42, np.nan, 36, 24, 73],
            'sex' : ['m', np.nan, 'f', 'm', 'f'],
            'preTestScore' : [4, np.nan, np.nan, 2, 3],
            'postTestScore' : [25, np.nan, np.nan, 62, 70]}

df = pd.DataFrame(raw_data, columns= ['first_name', 'last_name', 'age', 'sex', 'preTestScore', 'postTestScore'])
df

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [4]:
    # 결측치를 확인하기 위해 위와 같은 코드 사용

    # 먼저 isnull 함수를 사용
        # NaN값이 존재할 경우 True, 그렇지 않을 경우 False가 출력
            # 그리고 나서 sum 함수 사용 -> True인 경우를 모두 더함
                # 마지막으로 전체 데이터의 개수로 나눔
                    # 각 열별로 데이터의 결측치 비율이 나타남

df.isnull().sum() / len(df)
# 각 열별로 1, 2개의 결측치가 있어 20%, 40%가 출력됨

first_name       0.2
last_name        0.2
age              0.2
sex              0.2
preTestScore     0.4
postTestScore    0.4
dtype: float64

In [5]:
    # 1. 드롭
        # 결측치를 처리하는 첫 번째 전략은 드롭(drop), 즉 삭제
        # 앞의 데이터에서 NaN이 있는 모든 데이터의 행을 없애고자 dropna 함수 사용

df.dropna()
# 드롭과 관련된 대부분의 명령어들은 실제 드롭한 결과를 반환하는 함수
    # 하지만 현재 사용하는 객체에 드롭 결과를 저장하지는 않음
        # 따라서 inplace = True 또는 새로운 값에 복사해서 쓰면 됨

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [6]:
        # 어떤 조건을 입력하여 결측치를 지울 때는 매개변수 how를 사용
            # how에는 매개변수 all과 any 사용 가능
                # all : 행에 있는 모든 값이 NaN일 때 해당 행 삭제
                # any : 행에 하나라도 NaN이 있을 때 해당 행 삭제
            # dropna의 기본 설정은 any로 되어있음
                # df.dropna(how = 'all')로 입력하면 all 조건으로 바뀜

df_cleaned = df.dropna(how= 'all')
df_cleaned

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [7]:
        # 만약 열 값이 모두 NaN일 경우 축(axis)을 추가하여 삭제 가능

df['location'] = np.nan
df.dropna(axis= 1, how= 'all')
# location이라는 열 및 해당 값을 모두 NaN으로 추가한 후 해당 열만 삭제하는 코드
    # axis=1을 추가함으로써 열 기준으로 삭제 가능

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


In [8]:
        # 데이터의 개수를 기반으로 삭제도 가능
            # 매개변수 thresh 사용

df.dropna(axis= 0, thresh= 1)
# thresh=1을 사용하면 데이터가 한 개라도 존재하는 행은 그대로 남겨둠
    # thresh=2 면 두 개 이상 존재하는 행만 남겨둠
    

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
2,Tina,Ali,36.0,f,NaN,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


In [9]:
    # 2. 채우기
        # 채우기(fill)는 일반적으로 드롭(drop)한 후에 남은 값들을 어떻게 처리할 지 고민할 때 사용
        # 데이터를 채워 넣을 때 첫 번째로 생각할 것은 분포
            # 기본적으로 모든 데이터는 어떤 특정 분포를 갖고 있을 것
                # 이를 고려하여 비어있는 데이터 값을 채워야 함
        # 가장 쉬운 방법은 평균, 최빈값 등으로 채우는 것

df.fillna(0)
# 값을 채우기 위해서 fillna 함수 사용


,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,0.0
1,0,0,0.0,0,0.0,0.0,0.0
2,Tina,Ali,36.0,f,0.0,0.0,0.0
3,Jake,Milner,24.0,m,2.0,62.0,0.0
4,Amy,Cooze,73.0,f,3.0,70.0,0.0


In [11]:
df["preTestScore"].fillna(df["preTestScore"].mean(), inplace= True)
# 평균값을 채우는 코드
# inplace는 변경된 값을 리턴 시키는 것이 아닌 해당 변수 자체의 값을 변경시킴
df

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2,Tina,Ali,36.0,f,3.0,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


In [12]:
        # 필요에 따라 다른 열 분포를 고려하여 비어있는 값을 채울 수 있음
            # 남성과 여성의 성적에 대한 평균이 다르면 성별 별로 평균을 내어 값을 채울 수 있음

df.groupby("sex")[
    "postTestScore"].transform(
    "mean")
# groupby 함수를 사용하면 성별을 고려하여 평균값을 만들 수 있음
    # 각 인덱스의 성별에 따라 빈칸이 채워짐

0    43.5
1     NaN
2    70.0
3    43.5
4    70.0
Name: postTestScore, dtype: float64

In [13]:
df["postTestScore"].fillna(df.groupby("sex")["postTestScore"].transform("mean"), inplace= True)
df
# fillna 함수 안에 transform을 사용해 NaN이 있는 곳에 인덱스 기반으로 데이터를 채움
    # 이런 방식으로 대부분의 값을 채움

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,3.0,NaN,NaN
2,Tina,Ali,36.0,f,3.0,70.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


In [14]:
# 범주형 데이터 처리하기 : 원핫인코딩 (one-hot-encoding)
    # 범주형 데이터의 개수만큼 가변수(dummy variable)를 생성하여 존재 유무를 1 or 0으로 표현하는 기법
    # color 라는 변수에 다음과 같은 3개의 값이 있다고 가정
                # {Green, Blue, Yellow}
        # 이러한 세 가지 색깔이 있는 변수라면 다음과 같이 3개의 가변수를 만들고 각 색상에 인덱스를 지정
            # 그 후 해당 값이면 1, 아니면 0을 입력
                # Green : [1, 0, 0], Blue : [0, 1, 0], Yellow : [0, 0, 1]

In [15]:
    # 원핫인코딩을 적용하기 위한 방법
        # 1. 판다스에서 제공하는 get_dummies 함수 이용
        # 2. 사이킷런에서 제공하는 LabelEncoder, OneHotEncoder를 이용
        # 여기에선 get_dummies 사용

edges = pd.DataFrame({'source' : [0, 1, 2], 'target' : [2, 2, 3],
                      'weight' : [3, 4, 5], 'color' : ['red', 'blue', 'blue']})
edges

,source,target,weight,color
0,0,2,3,red
1,1,2,4,blue
2,2,3,5,blue


In [16]:
edges.dtypes
# 이 데이터의 dtypes를 보면 color의 타입이 object -> 범주형 데이터

source     int64
target     int64
weight     int64
color     object
dtype: object

In [21]:
    # get_dummies 적용

pd.get_dummies(edges, dtype= int)
# 범주형 데이터에 가변수 추가표시
# 드롭함수와 마찬가지로 실제 edges 변수는 변화가 없고 출력값만 생성됨

,source,target,weight,color_blue,color_red
0,0,2,3,0,1
1,1,2,4,1,0
2,2,3,5,1,0


In [24]:
    # 다른 방법으로 해당 열의 컬러 값만 따로 추출해서 적용할 수도 있음
        # 이 때 한 개의 열만 뽑는 결과와 이차원 행렬 형태로 데이터를 추출하는 것은 조금 다름
            # 한 개의 열만 추출하는 형태 : 시리즈 형태
            # 이차원 행렬 형태로 열 이름을 사용해 추출하는 형태 : 데이터프레임 형태
            # 위와 같이 추출되기 때문에 발생하는 현상

pd.get_dummies(edges['color'], dtype= int)

,blue,red
0,0,1
1,1,0
2,1,0


In [25]:
pd.get_dummies(edges[['color']], dtype= int)

,color_blue,color_red
0,0,1
1,1,0
2,1,0


In [26]:
    # 범주형 데이터를 처리할 때, 필요에 따라 정수형(int type)을 객체(object)로 변경해서 처리할 때가 있음

weight_dict = {3 : 'M', 4 : 'L', 5 : 'XL'}
edges["weight_sign"] = edges["weight"].map(weight_dict)
weight_sign = pd.get_dummies(edges["weight_sign"], dtype= int)
weight_sign
# weight는 숫자로 되어 있지만, 연속형 데이터가 아닌 기수형 데이터에 속함
    # 이럴 경우 각각의 데이터를 크기를 나타내는 것으로 변경하여 원핫인코딩을 적용할 필요가 있음
    

,L,M,XL
0,0,1,0
1,1,0,0
2,0,0,1


In [27]:
    # 데이터를 원핫인코딩 형태로 변경한 후
        # 필요에 따라 병합(merge) or 연결(concat)을 수행해 두 가지의 데이터를 합칠 필요가 있음

pd.concat([edges, weight_sign], axis= 1)
# 기존에 있던 edges와 weight_sign을 concat 함수로 통합

,source,target,weight,color,weight_sign,L,M,XL
0,0,2,3,red,M,0,1,0
1,1,2,4,blue,L,1,0,0
2,2,3,5,blue,XL,0,0,1


In [28]:
# 범주형 데이터로 변환하여 처리하기 : 바인딩(binding)
    # 기존 연속형 데이터를 범주형 데이터로 변환하여 처리해야할 때 사용
    # 바인딩 기법을 통해 범주형 데이터로 변환하여 처리할 때 나은 성능을 보여줌
        # 연속형 변수 자체에 의미가 있기 때문

raw_data = {'regiment' : ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks',
                          'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons',
                          'Scouts', 'Scouts', 'Scouts', 'Scouts'],
            'company' : ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd'],
            'name' : ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon',
                      'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'],
            'preTestScore' : [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
            'postTestScore' : [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}

df = pd.DataFrame(raw_data, columns= ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

,regiment,company,name,preTestScore,postTestScore
0,Nighthawks,1st,Miller,4,25
1,Nighthawks,1st,Jacobson,24,94
2,Nighthawks,2nd,Ali,31,57
3,Nighthawks,2nd,Milner,2,62
4,Dragoons,1st,Cooze,3,70
5,Dragoons,1st,Jacon,4,25
6,Dragoons,2nd,Ryaner,24,94
7,Dragoons,2nd,Sone,31,57
8,Scouts,1st,Sloan,2,62
9,Scouts,1st,Piger,3,70


In [31]:
    # 이 데이터에서 postTestScore의 점수에 대한 학점을 측정하는 코드를 작성할 때
        # 데이터에 대한 범위를 구분해야 함
            # cut 함수 사용
            

bins = [0, 25, 50, 75, 100]
group_names = ['D', 'C', 'B', 'A']
categories = pd.cut(df['postTestScore'], bins, labels= group_names)
categories
# bins의 원소 개수가 group_names의 원소 수보다 1개 더 많음
    # bins는 구간을 나타내기 때문에 (시작 값) - (끝 값) 형태로 구성되기 때문
# cut 함수로 나눌 시리즈 객체, 구간, 구간의 이름을 넣어줌

0     D
1     A
2     B
3     B
4     B
5     D
6     A
7     B
8     B
9     B
10    B
11    B
Name: postTestScore, dtype: category
Categories (4, object): ['D' < 'C' < 'B' < 'A']

In [32]:
# 데이터의 크기 맞추기 : 피쳐 스케일링(scaling)
    # 데이터의 크기를 맞춤
    # x1과 x2의 변수 범위가 다를 때 하나의 변수 범위로 통일시켜 처리해야 함
    # 일반적으로 스케일링을 위한 전략은 두 가지가 있음
        # 최솟값 - 최댓값 정규화
            # 최솟값과 최댓값을 기준으로 0에서 1 or 지정 값까지 값의 크기를 변화시키는 기법
                # 원래 값이 100에서 190이었다면 이를 0에서 1로 매핑하는 등의 기법
            # z1 = {(xi - min(x)) / (max(x) - min(x))} * (new_max - new_min) + new_min
                # x : 처리하고자 하는 열
                # xi : 이 열의 하나의 값
                # new_ : 새롭게 지정되는 범위의 값
        # z-스코어 정규화
            # 기존 값을 표준 정규분포값으로 변환하여 처리하는 기법
            # 가장 간단한 수식
                # z = (xi - μ) / σ
                    # μ : x열의 평균값
                    # σ : 표준편차

In [33]:
df = pd.DataFrame(
    {'A' : [14.00, 90.20, 90.95, 96.27, 91.21],
     'B' : [103.02, 107.26, 110.35, 114.23, 114.68],
     'C' : ['big', 'small', 'big', 'small', 'small']})

df
# 위 데이터에서 열 별로 스케일링을 할 예정

,A,B,C
0,14.00,103.02,big
1,90.20,107.26,small
2,90.95,110.35,big
3,96.27,114.23,small
4,91.21,114.68,small


In [34]:
    # 스케일링을 할 때 기본적으로 브로드캐스팅 개념이 사용됨
        # 평균값, 최댓값, 최솟값과 같은 스칼라값과 열과 같은 벡터 값들 간의 연산을 통해 앞의 수식을 연산

df['A'] - df['A'].min()
# 열 A에 전체적으로 최솟값을 빼기 위한 코드

0     0.00
1    76.20
2    76.95
3    82.27
4    77.21
Name: A, dtype: float64

In [35]:
    # 최솟값 - 최댓값 정규화 수식 사용

(df['A'] - df['A'].min()) / (df['A'].max() - df['A'].min())
# 새로운 범위가 0, 1일 때일 경우 위의 수식대로 함

0    0.000000
1    0.926219
2    0.935335
3    1.000000
4    0.938495
Name: A, dtype: float64

In [36]:
    # z - 스코어 정규화 수식 사용

(df['B'] - df['B'].mean()) / df['B'].std()

0   -1.405250
1   -0.540230
2    0.090174
3    0.881749
4    0.973556
Name: B, dtype: float64